코드 출처: https://github.com/deepseasw/seq2seq_chatbot<br>

In [1]:
from tensorflow import keras
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers, losses, metrics
from tensorflow.keras import preprocessing

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import os
import re

from konlpy.tag import Okt

<br>
<br>

# 데이터 로드

Seq2Seq에서의 임베딩이 이전 예제와 다른 점은 아래와 같이 태그를 사용한다는 것입니다.<br>
임베딩의 0~3번째에 각각 PADDING, START, END, OOV 태그를 넣습니다.<br>
사실 그냥 똑같은 단어라고 보시면 됩니다. 다만 이 단어들이 Seq2Seq의 동작을 제어합니다. <br>
<br>
예를 들어, 디코더 입력에 START가 들어가면 디코딩의 시작을 의미합니다. 반대로 디코더 출력에 END가 나오면 디코딩을 종료합니다.<br>

In [2]:
# 태그 단어
PAD = "<PADDING>"   # 패딩
STA = "<START>"     # 시작
END = "<END>"       # 끝
OOV = "<OOV>"       # 없는 단어(Out of Vocabulary)

# 태그 인덱스
PAD_INDEX = 0
STA_INDEX = 1
END_INDEX = 2
OOV_INDEX = 3

# 데이터 타입
ENCODER_INPUT  = 0
DECODER_INPUT  = 1
DECODER_TARGET = 2

# 한 문장에서 단어 시퀀스의 최대 개수
max_sequences = 30

# 임베딩 벡터 차원
embedding_dim = 100

# LSTM 히든레이어 차원
lstm_hidden_dim = 128

# 정규 표현식 필터
RE_FILTER = re.compile("[.,!?\"':;~()]")

# 챗봇 데이터 로드
chatbot_data = pd.read_csv('./dataset/chatbot/ChatbotData.csv', encoding='utf-8')
question, answer = list(chatbot_data['Q']), list(chatbot_data['A'])

챗봇의 훈련을 위해서 송영숙님이 공개한 한글 데이터셋을 로드합니다.<br>
질문과 대답, 감정 등 총 3개의 항목으로 구성되어 있습니다.<br>
감정 분류는 Seq2Seq에 필요가 없기 때문에 사용하지 않습니다.<br>
https://github.com/songys/Chatbot_data<br>


In [3]:
# 데이터 개수
len(question)


11824

In [4]:
# 데이터의 일부만 학습에 사용
question = question[:100]
answer = answer[:100]

# 챗봇 데이터 출력
for i in range(10):
    print('Q : ' + question[i])
    print('A : ' + answer[i])
    print()


Q : 12시 땡!
A : 하루가 또 가네요.

Q : 1지망 학교 떨어졌어
A : 위로해 드립니다.

Q : 3박4일 놀러가고 싶다
A : 여행은 언제나 좋죠.

Q : 3박4일 정도 놀러가고 싶다
A : 여행은 언제나 좋죠.

Q : PPL 심하네
A : 눈살이 찌푸려지죠.

Q : SD카드 망가졌어
A : 다시 새로 사는 게 마음 편해요.

Q : SD카드 안돼
A : 다시 새로 사는 게 마음 편해요.

Q : SNS 맞팔 왜 안하지ㅠㅠ
A : 잘 모르고 있을 수도 있어요.

Q : SNS 시간낭비인 거 아는데 매일 하는 중
A : 시간을 정하고 해보세요.

Q : SNS 시간낭비인데 자꾸 보게됨
A : 시간을 정하고 해보세요.



<br>
<br>

# 단어 사전 생성

In [5]:
# 형태소분석 함수
def pos_tag(sentences):
    
    # KoNLPy 형태소분석기 설정
    tagger = Okt()
    
    # 문장 품사 변수 초기화
    sentences_pos = []
    
    # 모든 문장 반복
    for sentence in sentences:
        # 특수기호 제거
        # RE_FILTER에 해당되는 정규표현식 char에 대하여 "" ()로 바꾸어라
        sentence = re.sub(RE_FILTER, "", sentence)
        
        # 배열인 형태소분석의 출력을 띄어쓰기로 구분하여 붙임
        sentence = " ".join(tagger.morphs(sentence))
        sentences_pos.append(sentence)
        
    return sentences_pos

In [6]:
# 형태소분석 수행
question = pos_tag(question)
answer = pos_tag(answer)

# 형태소분석으로 변환된 챗봇 데이터 출력
for i in range(10):
    print('Q : ' + question[i])
    print('A : ' + answer[i])
    print()

Q : 12시 땡
A : 하루 가 또 가네요

Q : 1 지망 학교 떨어졌어
A : 위로 해 드립니다

Q : 3 박 4일 놀러 가고 싶다
A : 여행 은 언제나 좋죠

Q : 3 박 4일 정도 놀러 가고 싶다
A : 여행 은 언제나 좋죠

Q : PPL 심하네
A : 눈살 이 찌푸려지죠

Q : SD 카드 망가졌어
A : 다시 새로 사는 게 마음 편해요

Q : SD 카드 안 돼
A : 다시 새로 사는 게 마음 편해요

Q : SNS 맞팔 왜 안 하지 ㅠㅠ
A : 잘 모르고 있을 수도 있어요

Q : SNS 시간 낭비 인 거 아는데 매일 하는 중
A : 시간 을 정 하고 해보세요

Q : SNS 시간 낭비 인데 자꾸 보게 됨
A : 시간 을 정 하고 해보세요



In [7]:
# 질문과 대답 문장들을 하나로 합침
sentences = []
sentences.extend(question)
sentences.extend(answer)

words = []

# 단어들의 배열 생성
for sentence in sentences:
    for word in sentence.split():
        words.append(word)

# 길이가 0인 단어는 삭제
words = [word for word in words if len(word) > 0]

# 중복된 단어 삭제
words = list(set(words))

# 제일 앞에 태그 단어 삽입
words[:0] = [PAD, STA, END, OOV]

질문과 대답 문장들을 합쳐서 전체 단어 사전을 만듭니다.<br>
자연어처리에서는 항상 이렇게 단어를 인덱스에 따라 정리를 해야 합니다.<br>
<br>
그래야지 문장을 인덱스 배열로 바꿔서 임베딩 레이어에 넣을 수 있습니다.<br>
또한 모델의 출력에서 나온 인덱스를 다시 단어로 변환하는데도 필요합니다.<br>
<br>

In [8]:
# 단어 개수
len(words)

454

In [9]:
# 단어 출력
words

['<PADDING>',
 '<START>',
 '<END>',
 '<OOV>',
 '쇼핑',
 '서로',
 '달',
 '건데',
 '이나',
 '되',
 '변화',
 '즐거운',
 '생각',
 '콕',
 '벗어나는',
 '마세요',
 '있어요',
 '하',
 '누굴',
 '가지',
 '사세요',
 '일',
 '지망',
 '매일',
 '갈거야',
 '인게',
 '심하네',
 '다음',
 '엉망',
 '무시',
 '됐으면',
 '했어',
 '괜찮아요',
 '후회',
 '풀었어',
 '아님',
 '남겨야',
 '해보세요',
 '있어',
 '힘든데',
 '있을까',
 '식혀주세요',
 '좋아요',
 '옴',
 '간다',
 '걸린',
 '듣고',
 '그럴',
 '1',
 '없어',
 '먹었는데',
 '드립니다',
 '보내고',
 '절약',
 '간장',
 '말',
 '개념',
 '처럼',
 '사이',
 '키워',
 '말랭이',
 '같아',
 '마음',
 '좋은',
 '그건',
 '감미로운',
 '감기',
 '떨리니까',
 '역시',
 '편해요',
 '알려',
 '알',
 '첫인상',
 '그',
 '싶은데',
 '시켜',
 '좋다',
 '자의',
 '개강',
 '누구',
 '일도',
 '해봐요',
 '야',
 '좋겠다',
 '낮잠',
 '진창',
 '입어볼까',
 '기름',
 '부터는',
 '돼겠지',
 '쉬는',
 '추천',
 '갑자기',
 '살쪄도',
 '출발',
 '될',
 '좋죠',
 '나쁜',
 '사는',
 '왜',
 '연인',
 '이에요',
 '상황',
 '나갔어',
 '말까',
 '물어',
 '가네요',
 '가스',
 '어필',
 '데이터',
 '불',
 '책임질',
 '하루',
 '있어도',
 '당황',
 '뭘',
 '중',
 '니까',
 'PPL',
 '보고싶었나',
 '하는',
 '까',
 '기관',
 '불편한',
 '하겠어',
 '하는데',
 '만',
 '을',
 '아름다운',
 'ㅠㅠ',
 '3초',
 '됨',
 '켜놓고',
 '세수',
 '싫어',

In [10]:
# 단어와 인덱스의 딕셔너리 생성, Series.count_values()도 가능
word_to_index = {word: index for index, word in enumerate(words)}
index_to_word = {index: word for index, word in enumerate(words)}

In [11]:
# 단어 -> 인덱스
# 문장을 인덱스로 변환하여 모델 입력으로 사용
word_to_index

{'<PADDING>': 0,
 '<START>': 1,
 '<END>': 2,
 '<OOV>': 3,
 '쇼핑': 4,
 '서로': 5,
 '달': 6,
 '건데': 7,
 '이나': 8,
 '되': 9,
 '변화': 10,
 '즐거운': 11,
 '생각': 12,
 '콕': 13,
 '벗어나는': 14,
 '마세요': 15,
 '있어요': 16,
 '하': 17,
 '누굴': 18,
 '가지': 19,
 '사세요': 20,
 '일': 21,
 '지망': 22,
 '매일': 23,
 '갈거야': 24,
 '인게': 25,
 '심하네': 26,
 '다음': 27,
 '엉망': 28,
 '무시': 29,
 '됐으면': 30,
 '했어': 31,
 '괜찮아요': 32,
 '후회': 33,
 '풀었어': 34,
 '아님': 35,
 '남겨야': 36,
 '해보세요': 37,
 '있어': 38,
 '힘든데': 39,
 '있을까': 40,
 '식혀주세요': 41,
 '좋아요': 42,
 '옴': 43,
 '간다': 44,
 '걸린': 45,
 '듣고': 46,
 '그럴': 47,
 '1': 48,
 '없어': 49,
 '먹었는데': 50,
 '드립니다': 51,
 '보내고': 52,
 '절약': 53,
 '간장': 54,
 '말': 55,
 '개념': 56,
 '처럼': 57,
 '사이': 58,
 '키워': 59,
 '말랭이': 60,
 '같아': 61,
 '마음': 62,
 '좋은': 63,
 '그건': 64,
 '감미로운': 65,
 '감기': 66,
 '떨리니까': 67,
 '역시': 68,
 '편해요': 69,
 '알려': 70,
 '알': 71,
 '첫인상': 72,
 '그': 73,
 '싶은데': 74,
 '시켜': 75,
 '좋다': 76,
 '자의': 77,
 '개강': 78,
 '누구': 79,
 '일도': 80,
 '해봐요': 81,
 '야': 82,
 '좋겠다': 83,
 '낮잠': 84,
 '진창': 85,
 '입어볼까': 86,
 '기름': 8

In [12]:
# 인덱스 -> 단어
# 모델의 예측 결과인 인덱스를 문장으로 변환시 사용
index_to_word


{0: '<PADDING>',
 1: '<START>',
 2: '<END>',
 3: '<OOV>',
 4: '쇼핑',
 5: '서로',
 6: '달',
 7: '건데',
 8: '이나',
 9: '되',
 10: '변화',
 11: '즐거운',
 12: '생각',
 13: '콕',
 14: '벗어나는',
 15: '마세요',
 16: '있어요',
 17: '하',
 18: '누굴',
 19: '가지',
 20: '사세요',
 21: '일',
 22: '지망',
 23: '매일',
 24: '갈거야',
 25: '인게',
 26: '심하네',
 27: '다음',
 28: '엉망',
 29: '무시',
 30: '됐으면',
 31: '했어',
 32: '괜찮아요',
 33: '후회',
 34: '풀었어',
 35: '아님',
 36: '남겨야',
 37: '해보세요',
 38: '있어',
 39: '힘든데',
 40: '있을까',
 41: '식혀주세요',
 42: '좋아요',
 43: '옴',
 44: '간다',
 45: '걸린',
 46: '듣고',
 47: '그럴',
 48: '1',
 49: '없어',
 50: '먹었는데',
 51: '드립니다',
 52: '보내고',
 53: '절약',
 54: '간장',
 55: '말',
 56: '개념',
 57: '처럼',
 58: '사이',
 59: '키워',
 60: '말랭이',
 61: '같아',
 62: '마음',
 63: '좋은',
 64: '그건',
 65: '감미로운',
 66: '감기',
 67: '떨리니까',
 68: '역시',
 69: '편해요',
 70: '알려',
 71: '알',
 72: '첫인상',
 73: '그',
 74: '싶은데',
 75: '시켜',
 76: '좋다',
 77: '자의',
 78: '개강',
 79: '누구',
 80: '일도',
 81: '해봐요',
 82: '야',
 83: '좋겠다',
 84: '낮잠',
 85: '진창',
 86: '입어볼까',
 87: '기름

<br>
<br>

# 전처리

In [13]:
# 문장을 인덱스로 변환
def convert_text_to_index(sentences, vocabulary, type): 
    
    sentences_index = []
    
    # 모든 문장에 대해서 반복
    for sentence in sentences:
        sentence_index = []
        
        # 디코더 입력일 경우 맨 앞에 START 태그 추가
        if type == DECODER_INPUT:
            sentence_index.extend([vocabulary[STA]])
        
        # 문장의 단어들을 띄어쓰기로 분리
        for word in sentence.split():
            if vocabulary.get(word) is not None:
                # 사전에 있는 단어면 해당 인덱스를 추가
                sentence_index.extend([vocabulary[word]])
            else:
                # 사전에 없는 단어면 OOV 인덱스를 추가
                sentence_index.extend([vocabulary[OOV]])

        # 최대 길이 검사
        if type == DECODER_TARGET:
            # 디코더 목표일 경우 맨 뒤에 END 태그 추가
            if len(sentence_index) >= max_sequences:
                sentence_index = sentence_index[:max_sequences-1] + [vocabulary[END]]
            else:
                sentence_index += [vocabulary[END]]
        else:
            if len(sentence_index) > max_sequences:
                sentence_index = sentence_index[:max_sequences]
            
        # 최대 길이에 없는 공간은 패딩 인덱스로 채움
        sentence_index += (max_sequences - len(sentence_index)) * [vocabulary[PAD]]
        
        # 문장의 인덱스 배열을 추가
        sentences_index.append(sentence_index)

    return np.asarray(sentences_index)

Seq2Seq에서는 학습시 다음과 같이 총 3개의 데이터가 필요합니다.

인코더 입력 : 12시 땡<br>
디코더 입력 : START 하루 가 또 가네요<br>
디코더 출력 : 하루 가 또 가네요 END
<br>
원래 Seq2Seq는 디코더의 현재 출력이 디코더의 다음 입력으로 들어갑니다.<br>
다만 학습에서는 굳이 이렇게 하지 않고 디코더 입력과 디코더 출력의 데이터를 각각 만듭니다. <br>
<br>
그러나 예측시에는 이런 방식이 불가능합니다.<br>
출력값을 미리 알지 못하기 때문에, 디코더 입력을 사전에 생성할 수가 없습니다.<br>
이런 문제를 해결하기 위해 훈련 모델과 예측 모델을 따로 구성해야 합니다.<br>
모델 생성 부분에서 다시 자세히 설명을 드리겠습니다.<br>
<br>
<br>

In [14]:
# 인코더 입력 인덱스 변환
x_encoder = convert_text_to_index(question, word_to_index, ENCODER_INPUT)

print('첫번째 question:', question[0])
# 첫 번째 인코더 입력 출력 (12시 땡)
print('첫번째 Encoder 입력:',x_encoder[0])
print()

# 출력된 125, 308은 word의 index
print('index_to_word[ 125 ]:', index_to_word[ 125 ])
print('index_to_word[ 308 ]:', index_to_word[ 308 ])

첫번째 question: 12시 땡
첫번째 Encoder 입력: [243 154   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0]

index_to_word[ 125 ]: 하는데
index_to_word[ 308 ]: 정


In [15]:
# 디코더 입력 인덱스 변환
x_decoder = convert_text_to_index(answer, word_to_index, DECODER_INPUT)

print('첫번째 answer:', answer[0])
print('첫번째 Decoder input:', x_decoder[0])
print()

# 출력된 1, 259, 223, 114, 74는 word의 index
print('index_to_word[   1 ]:', index_to_word[   1 ])
print('index_to_word[ 259 ]:', index_to_word[ 259 ])
print('index_to_word[ 223 ]:', index_to_word[ 223 ])
print('index_to_word[ 114 ]:', index_to_word[ 114 ])
print('index_to_word[  74 ]:', index_to_word[  74 ])

첫번째 answer: 하루 가 또 가네요
첫번째 Decoder input: [  1 112 452 196 106   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0]

index_to_word[   1 ]: <START>
index_to_word[ 259 ]: 만나지
index_to_word[ 223 ]: 예쁘게
index_to_word[ 114 ]: 당황
index_to_word[  74 ]: 싶은데


In [16]:
# 디코더 목표 인덱스 변환
y_decoder = convert_text_to_index(answer, word_to_index, DECODER_TARGET)

# 첫 번째 디코더 목표 출력 (하루 가 또 가네요 END)
y_decoder[0]

print('첫번째 answer:', answer[0])
print('첫번쨰 Decoder onput:', y_decoder[0])
print()

# 출력된 1, 259, 223, 114, 74는 word의 index
print('index_to_word[ 259 ]:', index_to_word[ 259 ])
print('index_to_word[ 223 ]:', index_to_word[ 223 ])
print('index_to_word[ 114 ]:', index_to_word[ 114 ])
print('index_to_word[  74 ]:', index_to_word[  74 ])
print('index_to_word[   2 ]:', index_to_word[   2 ])

첫번째 answer: 하루 가 또 가네요
첫번쨰 Decoder onput: [112 452 196 106   2   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0]

index_to_word[ 259 ]: 만나지
index_to_word[ 223 ]: 예쁘게
index_to_word[ 114 ]: 당황
index_to_word[  74 ]: 싶은데
index_to_word[   2 ]: <END>


In [17]:
# 원핫인코딩 초기화
one_hot_data = np.zeros((len(y_decoder), max_sequences, len(words)))
one_hot_data[0]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [18]:
#one_hot_data의 첫 row에 259번째 column에 '하루'
#one_hot_data의 첫 row에 223번째 column에 '가'
#one_hot_data의 첫 row에 114번째 column에 '또'.... 를 1로 변경
y_decoder[0]

array([112, 452, 196, 106,   2,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0])

In [19]:
# 디코더 목표를 원핫인코딩으로 변환
# 학습시 입력은 인덱스이지만, 출력은 원핫인코딩 형식임
for i, sequence in enumerate(y_decoder):
    for j, index in enumerate(sequence):
        one_hot_data[i, j, index] = 1

# 디코더 목표 설정
y_decoder = one_hot_data

# 첫 번째 디코더 목표 출력
y_decoder[0]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]])

In [32]:
# 259번째 index (하루) 가 1임
y_decoder[0][0][112]

1.0

인코더 입력과 디코더 입력은 임베딩 레이어에 들어가는 인덱스 배열입니다.<br>
반면에 디코더 출력은 원핫인코딩 형식이 되어야 합니다.<br>
디코더의 마지막 Dense 레이어에서 softmax로 나오기 때문입니다.<br>

<br>
<br>

# 모델 생성

In [21]:
#--------------------------------------------
# 훈련 모델 인코더 정의
#--------------------------------------------

# 입력 문장의 인덱스 시퀀스를 입력으로 받음
encoder_inputs = layers.Input(shape=(None,))

# 임베딩 레이어
encoder_outputs = layers.Embedding(len(words), embedding_dim)(encoder_inputs)

# return_state가 True면 상태값 리턴
# LSTM은 state_h(hidden state)와 state_c(cell state) 2개의 상태 존재
encoder_outputs, state_h, state_c = layers.LSTM(lstm_hidden_dim,
                                                dropout=0.1,
                                                recurrent_dropout=0.5,
                                                return_state=True)(encoder_outputs)

# 히든 상태와 셀 상태를 하나로 묶음
# Decoder의 initial state에 넣어주기 위함
# 즉, input sentence의 모든 정보를 통해 Decoding 하기 위함
encoder_states = [state_h, state_c]


In [31]:
encoder_outputs

<KerasTensor: shape=(None, 128) dtype=float32 (created by layer 'lstm')>

In [22]:
encoder_states

[<KerasTensor: shape=(None, 128) dtype=float32 (created by layer 'lstm')>,
 <KerasTensor: shape=(None, 128) dtype=float32 (created by layer 'lstm')>]

In [23]:
#--------------------------------------------
# 훈련 모델 디코더 정의
#--------------------------------------------

# 목표 문장의 인덱스 시퀀스를 입력으로 받음
decoder_inputs = layers.Input(shape=(None,))

# 임베딩 레이어
decoder_embedding = layers.Embedding(len(words), embedding_dim)
decoder_outputs = decoder_embedding(decoder_inputs)

# 인코더와 달리 return_sequences를 True로 설정하여 모든 타임 스텝 출력값 리턴
# 모든 타임 스텝의 출력값들을 다음 레이어의 Dense()로 처리하기 위함
decoder_lstm = layers.LSTM(lstm_hidden_dim,
                           dropout=0.1,
                           recurrent_dropout=0.5,
                           return_state=True,
                           return_sequences=True)

# initial_state를 인코더의 상태로 초기화
decoder_outputs, _, _ = decoder_lstm(decoder_outputs,
                                     initial_state=encoder_states)

# 단어의 개수만큼 노드의 개수를 설정하여 원핫 형식으로 각 단어 인덱스를 출력
decoder_dense = layers.Dense(len(words), activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)


지금까지의 예제는 Sequential 방식의 모델이었습니다.<br>
하지만 이번에는 함수형 API 모델을 사용했습니다.<br>
인코더와 디코더가 따로 분리되어야 하는데, 단순히 레이어를 추가하여 붙이는 순차형으로는 구현이 불가능하기 때문입니다. <br>
<br>


In [24]:
#--------------------------------------------
# 훈련 모델 정의
#--------------------------------------------

# 입력과 출력으로 함수형 API 모델 생성
model = models.Model([encoder_inputs, decoder_inputs], decoder_outputs)

# 학습 방법 설정
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['acc'])    

Model() 함수로 입력과 출력을 따로 설정하여 모델을 만듭니다.<br>
그다음 compile과 fit은 이전과 동일하게 적용하시면 됩니다.<br>
<br>
<br>

In [25]:
#--------------------------------------------
#  예측 모델 인코더 정의
#--------------------------------------------

# 훈련 모델의 인코더 상태를 사용하여 예측 모델 인코더 설정
encoder_model = models.Model(encoder_inputs, encoder_states)

#--------------------------------------------
# 예측 모델 디코더 정의
#--------------------------------------------

# 예측시에는 훈련시와 달리 타임 스텝을 한 단계씩 수행
# 매번 이전 디코더 상태를 입력으로 받아서 새로 설정
decoder_state_input_h = layers.Input(shape=(lstm_hidden_dim,))
decoder_state_input_c = layers.Input(shape=(lstm_hidden_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]    

# 임베딩 레이어
decoder_outputs = decoder_embedding(decoder_inputs)

# LSTM 레이어
decoder_outputs, state_h, state_c = decoder_lstm(decoder_outputs,
                                                 initial_state=decoder_states_inputs)

# 히든 상태와 셀 상태를 하나로 묶음
decoder_states = [state_h, state_c]

# Dense 레이어를 통해 원핫 형식으로 각 단어 인덱스를 출력
decoder_outputs = decoder_dense(decoder_outputs)

# 예측 모델 디코더 설정
decoder_model = models.Model([decoder_inputs] + decoder_states_inputs,
                      [decoder_outputs] + decoder_states)

예측 모델은 이미 학습된 훈련 모델의 레이어들을 그대로 재사용합니다. 예측 모델 인코더는 훈련 모델 인코더과 동일합니다. 그러나 예측 모델 디코더는 매번 LSTM 상태값을 입력으로 받습니다. 또한 디코더의 LSTM 상태를 출력값과 같이 내보내서, 다음 번 입력에 넣습니다. 

이렇게 하는 이유는 LSTM을 딱 한번의 타임 스텝만 실행하기 때문입니다. 그래서 매번 상태값을 새로 초기화 해야 합니다. 이와 반대로 훈련할때는 문장 전체를 계속 LSTM으로 돌리기 때문에 자동으로 상태값이 전달됩니다. 

In [26]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, None, 100)    45400       input_1[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 100)    45400       input_2[0][0]                    
______________________________________________________________________________________________

In [27]:
encoder_model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None)]            0         
_________________________________________________________________
embedding (Embedding)        (None, None, 100)         45400     
_________________________________________________________________
lstm (LSTM)                  [(None, 128), (None, 128) 117248    
Total params: 162,648
Trainable params: 162,648
Non-trainable params: 0
_________________________________________________________________


In [28]:
# Decoder_model에는 
decoder_model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 100)    45400       input_2[0][0]                    
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 128)]        0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, 128)]        0                                            
____________________________________________________________________________________________

<br>
<br>

# 훈련 및 테스트

In [29]:
# 인덱스를 문장으로 변환
def convert_index_to_text(indexs, vocabulary): 
    
    sentence = ''
    
    # 모든 문장에 대해서 반복
    for index in indexs:
        if index == END_INDEX:
            # 종료 인덱스면 중지
            break;
        elif vocabulary.get(index) is not None:
            # 사전에 있는 인덱스면 해당 단어를 추가
            sentence += vocabulary[index]
        else:
            # 사전에 없는 인덱스면 OOV 단어를 추가
            sentence += vocabulary[OOV_INDEX]
            
        # 빈칸 추가
        sentence += ' '

    return sentence

In [30]:
# 에폭 반복
for epoch in range(20):
    print('Total Epoch :', epoch + 1)

    # 훈련 시작
    history = model.fit([x_encoder, x_decoder],
                        y_decoder,
                        epochs=100,
                        batch_size=64,
                        verbose=0)
    
    # 정확도와 손실 출력
    print('accuracy :', history.history['acc'][-1])
    print('loss :', history.history['loss'][-1])
    
    # 문장 예측 테스트
    # (3 박 4일 놀러 가고 싶다) -> (여행 은 언제나 좋죠)
    input_encoder = x_encoder[2].reshape(1, x_encoder[2].shape[0])
    input_decoder = x_decoder[2].reshape(1, x_decoder[2].shape[0])
    results = model.predict([input_encoder, input_decoder])
    
    # 결과의 원핫인코딩 형식을 인덱스로 변환
    # 1축을 기준으로 가장 높은 값의 위치를 구함
    indexs = np.argmax(results[0], 1) 
    
    # 인덱스를 문장으로 변환
    sentence = convert_index_to_text(indexs, index_to_word)
    print(sentence)
    print()


Total Epoch : 1
accuracy : 0.8396666646003723
loss : 0.6992223262786865
저 도 이 요 

Total Epoch : 2
accuracy : 0.925000011920929
loss : 0.34692755341529846
저 은 에는 좋죠 

Total Epoch : 3
accuracy : 0.9683333039283752
loss : 0.14421546459197998
저 은 언제나 좋죠 

Total Epoch : 4
accuracy : 0.9746666550636292
loss : 0.08817499876022339
저 은 언제나 좋죠 

Total Epoch : 5
accuracy : 0.9769999980926514
loss : 0.06913413852453232
여행 은 언제나 좋죠 

Total Epoch : 6
accuracy : 0.9823333621025085
loss : 0.04941220581531525
여행 은 언제나 좋죠 

Total Epoch : 7
accuracy : 0.9893333315849304
loss : 0.03170783445239067
여행 은 언제나 좋죠 

Total Epoch : 8
accuracy : 0.9929999709129333
loss : 0.02188185416162014
여행 은 언제나 좋죠 

Total Epoch : 9
accuracy : 0.9976666569709778
loss : 0.01334693469107151
여행 은 언제나 좋죠 

Total Epoch : 10
accuracy : 0.9983333349227905
loss : 0.006254453677684069
여행 은 언제나 좋죠 

Total Epoch : 11
accuracy : 0.9993333220481873
loss : 0.0033200467005372047
여행 은 언제나 좋죠 

Total Epoch : 12
accuracy : 0.9993333220481873
l

학습이 진행될수록 예측 문장이 제대로 생성되는 것을 볼 수 있습니다. 다만 여기서의 예측은 단순히 테스트를 위한 것이라, 인코더 입력과 디코더 입력 데이터가 동시에 사용됩니다. 아래 문장 생성에서는 예측 모델을 적용하기 때문에, 오직 인코더 입력 데이터만 집어 넣습니다.
<br>
<br>
<br>

In [33]:
# 모델 저장
encoder_model.save('./model/seq2seq_chatbot_encoder_model.h5')
decoder_model.save('./model/seq2seq_chatbot_decoder_model.h5')

# 인덱스 저장
with open('./model/word_to_index.pkl', 'wb') as f:
    pickle.dump(word_to_index, f, pickle.HIGHEST_PROTOCOL)
with open('./model/index_to_word.pkl', 'wb') as f:
    pickle.dump(index_to_word, f, pickle.HIGHEST_PROTOCOL)    

<br>
<br>

# 문장 생성

In [34]:
# 모델 파일 로드
encoder_model = models.load_model('./model/seq2seq_chatbot_encoder_model.h5')
decoder_model = models.load_model('./model/seq2seq_chatbot_decoder_model.h5')

# 인덱스 파일 로드
with open('./model/word_to_index.pkl', 'rb') as f:
    word_to_index = pickle.load(f)
with open('./model/index_to_word.pkl', 'rb') as f:
    index_to_word = pickle.load(f)    

In [35]:
# 예측을 위한 입력 생성
def make_predict_input(sentence):

    sentences = []
    sentences.append(sentence)
    sentences = pos_tag(sentences)
    input_seq = convert_text_to_index(sentences, word_to_index, ENCODER_INPUT)
    
    return input_seq

In [36]:
# 텍스트 생성
def generate_text(input_seq):
    
    # 입력을 인코더에 넣어 마지막 상태 구함
    states = encoder_model.predict(input_seq)

    # 목표 시퀀스 초기화
    target_seq = np.zeros((1, 1))
    
    # 목표 시퀀스의 첫 번째에 <START> 태그 추가
    target_seq[0, 0] = STA_INDEX
    
    # 인덱스 초기화
    indexs = []
    
    # 디코더 타임 스텝 반복
    while 1:
        # 디코더로 현재 타임 스텝 출력 구함
        # 처음에는 인코더 상태를, 다음부터 이전 디코더 상태로 초기화
        decoder_outputs, state_h, state_c = decoder_model.predict(
                                                [target_seq] + states)

        # 결과의 원핫인코딩 형식을 인덱스로 변환
        index = np.argmax(decoder_outputs[0, 0, :])
        indexs.append(index)
        
        # 종료 검사
        if index == END_INDEX or len(indexs) >= max_sequences:
            break

        # 목표 시퀀스를 바로 이전의 출력으로 설정
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = index
        
        # 디코더의 이전 상태를 다음 디코더 예측에 사용
        states = [state_h, state_c]

    # 인덱스를 문장으로 변환
    sentence = convert_index_to_text(indexs, index_to_word)
        
    return sentence

제일 첫 단어는 START로 시작합니다. 그리고 출력으로 나온 인덱스를 디코더 입력으로 넣고 다시 예측을 반복합니다. 상태값을 받아 다시 입력으로 같이 넣는 것에 주의하시기 바랍니다. END 태그가 나오면 문장 생성을 종료합니다.
<br>
<br>
<br>

In [42]:
# 문장을 인덱스로 변환
input_seq = make_predict_input('휴강이 좋아요')
input_seq


array([[  3, 412,  42,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0]])

In [43]:
# 예측 모델로 텍스트 생성
sentence = generate_text(input_seq)
sentence


'병원 가세 요 '

데이터셋에 있는 문장과 똑같은 입력을 넣으니, 역시 정확히 일치하는 답변이 출력되었습니다.
<br>
<br>
<br>

In [44]:
# 문장을 인덱스로 변환
input_seq = make_predict_input('데이터 강의 너무 좋아요')
input_seq


array([[109, 269, 232,  42,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0]])

In [45]:
# 예측 모델로 텍스트 생성
sentence = generate_text(input_seq)
sentence


'최고 의 강의 에요 '

최고의 강의입니다
<br>
<br>
<br>

In [51]:
# 문장을 인덱스로 변환
input_seq = make_predict_input('4박5일 욜로가려고요')
input_seq

array([[  3, 399,   3,   3, 143,   3,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0]])

In [52]:
# 예측 모델로 텍스트 생성
sentence = generate_text(input_seq)
sentence

'가세 요 '

하지만 데이터셋에 없던 '4박5일, 욜로'로 입력을 수정하니, 전혀 다른 문장이 출력되었습니다.<br>
이는 우리가 데이터의 일부인 100개 문장만 학습했기 때문입니다.<br>
데이터의 개수를 늘려서 훈련할수록 일반화 능력이 더욱 높아집니다.